In [1]:
# Import pandas & yfinance
import pandas as pd
import yfinance as yf

# Import numpy
import numpy as np
from numpy import *
from numpy.linalg import multi_dot

# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000,600))) # theme= 'henanigans'

# Import plotly express for EF plot
import plotly.express as px
# px.defaults.template = "plotly_dark"
px.defaults.width, px.defaults.height = 1000, 600

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)

In [22]:
# Specify assets / stocks
# international etf portfolio : 'SPY', 'GLD', 'IWM', 'VWO', 'BND']
# indian stocks : bank, consumer goods, diversified, it, consumer durables
assets = ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS']
assets.sort()

# Number of assets
numofasset = len(assets)

# Number of portfolio for optimization
numofportfolio = 25000

In [3]:
mu = np.array([
        [0.05], 
        [0.07], 
        [0.15], 
        [0.27],
    ])

sigma = np.array([
        [0.07], 
        [0.12], 
        [0.3], 
        [0.6],
    ])

rho = np.array([   
        [1, 0.8, 0.5, 0.4], 
        [0.8, 1, 0.7, 0.5], 
        [0.5, 0.7, 1, 0.8], 
        [0.4, 0.5, 0.8, 1]
    ])
cov = np.diag(sigma.flatten()) @ rho @ np.diag(sigma.flatten())

In [29]:
def portfolio_simulation(mu, sigma, cov):

    # Initialize the lists
    rets = []; vols = []; wts = []

    # Simulate 5,000 portfolios
    for i in range (numofportfolio):
        
        # Generate random weights
        weights = random.random(numofasset)[:, newaxis]
        
        # Set weights such that sum of weights equals 1
        weights /= sum(weights)
        
        # Portfolio statistics
        rets.append(weights.T @ mu)        
        vols.append(sqrt(multi_dot([weights.T, cov, weights])))
        wts.append(weights.flatten())

    # Create a dataframe for analysis
    portdf = 100*pd.DataFrame({
        'port_rets': array(rets).flatten(),
        'port_vols': array(vols).flatten(),
        'weights': list(array(wts))
        })
    
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']

    return round(portdf,2)


In [30]:
# Create a dataframe for analysis
temp = portfolio_simulation(mu, sigma, cov)
temp.head()

,port_rets,port_vols,weights,sharpe_ratio
0,11.50,19.78,"[37.341414110774515, 13.482776881799733, 38.25...",0.58
1,11.78,20.32,"[45.710169986129564, 6.460365995844028, 32.287...",0.58
2,11.70,20.42,"[16.303402793494854, 34.1878751311553, 40.6539...",0.57
3,10.47,17.28,"[58.60980014914071, 1.216396833144163, 28.2920...",0.61
4,13.13,23.51,"[21.383348651289346, 23.326559882483224, 37.51...",0.56


In [31]:
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets                                                    5.75
port_vols                                                    7.81
weights         [92.24342730864053, 3.129518140489652, 2.79608...
sharpe_ratio                                                 0.74
Name: 19513, dtype: object

In [32]:
# Verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,25000.0,13.5028,2.7911,5.50,11.52,13.50,15.33,25.36
port_vols,25000.0,24.6717,6.6729,7.54,19.85,24.44,28.95,55.55
sharpe_ratio,25000.0,0.5575,0.0394,0.46,0.53,0.55,0.58,0.74


In [33]:
# Max sharpe ratio portfolio weights
msrpwts = temp['weights'][temp['sharpe_ratio'].idxmax()]

# Allocation to achieve max sharpe ratio portfolio
dict(zip(assets, around(msrpwts,2)))

{'HDFCBANK': 92.24, 'ITC': 3.13, 'RELIANCE': 2.8, 'TCS': 1.83}

In [34]:
# Plot simulated portfolio
fig = px.scatter(
    temp, x='port_vols', y='port_rets', color='sharpe_ratio', 
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'},
    title="Monte Carlo Simulated Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot max sharpe 
fig.add_scatter(
    mode='markers', 
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']], 
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']], 
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [16]:
# Import optimization module from scipy
import scipy.optimize as sco

In [17]:
def portfolio_stats(weights):
    
    weights = array(weights)[:,newaxis]
    port_rets = weights.T @ array(returns.mean() * 252)[:,newaxis]    
    port_vols = sqrt(multi_dot([weights.T, returns.cov() * 252, weights])) 
    
    return np.array([port_rets, port_vols, port_rets/port_vols]).flatten()


# Minimize the volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

# Maximizing sharpe ratio
def max_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

In [18]:
# Specify constraints, bounds and initial weights
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0, 1) for x in range(numofasset))
initial_wts = numofasset*[1./numofasset]

In [19]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(max_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

# Optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

In [20]:
# Efficient Frontier
# targetrets = linspace(0.01,0.11,100)
targetrets = linspace(0.155,0.24,100)
tvols = []

for tr in targetrets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr},
               {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons)
    
    tvols.append(opt_ef['fun'])

targetvols = array(tvols)

In [21]:
# Create EF Dataframe for plotting
efport = pd.DataFrame({
    'targetrets' : around(100*targetrets,2),
    'targetvols': around(100*targetvols,2),
    'targetsharpe': around(targetrets/targetvols,2)
})

efport.head()

,targetrets,targetvols,targetsharpe
0,15.50,17.15,0.90
1,15.59,17.09,0.91
2,15.67,17.04,0.92
3,15.76,16.99,0.93
4,15.84,16.94,0.94


In [22]:
# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets',  color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected Volatility','targetsharpe': 'Sharpe Ratio'},
    title="Efficient Frontier Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))


# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_sharpe['x'])[1]], 
    y=[100*portfolio_stats(opt_sharpe['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]], 
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name = 'Min Variance'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()